In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import kerastuner as kt
from sklearn.model_selection import train_test_split

def plot_curve(epochs, hist, list_of_metrics,name, title, label_list):
    """Plot a curve of one or more classification metrics vs. epoch."""  
    plt.figure()
    plt.title(title)
    plt.xlabel("Epoch "+name)
    plt.ylabel("Value")

    for (m,label) in zip(list_of_metrics, label_list):
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=label)
    plt.legend()

      
# for activation functions check https://keras.io/api/layers/activations/
def create_model2(my_learning_rate,layers, my_feature_layer,my_metrics,my_act_function = "softmax"):
    model = tf.keras.models.Sequential()
    model.add(my_feature_layer)

    for layer in layers:
        model.add(tf.keras.layers.Dense(units = layer, activation = my_act_function))
    model.add(tf.keras.layers.Dense(units=1,name='Output', activation = 'relu'))                             
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=my_learning_rate,momentum=0.2),                                       
                loss=tf.keras.losses.MeanAbsoluteError(),
                metrics=my_metrics)
    return model


def train_model(model, dataset, epochs, label_name,
                batch_size=None,shuffle=True):
    features = {name:np.array(value) for name, value in dataset.items()}
    label=dataset[label_name].to_numpy()
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle,validation_split=0.2, verbose =0)
  
    epochs = history.epoch
    hist = pd.DataFrame(history.history)
    return epochs, hist
    
    
#returns dataframe
def test_model(model,dataset,label_name):
    features = {name:np.array(value) for name, value in dataset.items()}
    label=dataset[label_name].to_numpy()
    print(label)
    evaluation=model.evaluate(x = features, y = label, batch_size=batch_size, verbose = 0)
    predicted = model.predict(features)
    
    df_test=pd.DataFrame(label,columns=[label_name])

    df_predict=pd.DataFrame(predicted,columns=[label_name+"_pred"])
    return pd.concat([df_test,df_predict], axis=1)



In [3]:
#G:\OneDrive - bwedu\Master\Forschungspraktikum\Inga\pc-forschi\generated_Data
df_lambda=pd.read_csv("../../generated_Data/lambda_100_steps.csv")[["lambda","main_maximum"]]
df_delta=pd.read_csv("../../generated_Data/delta_100_steps.csv")[["delta","main_maximum"]]


lambda_train, lambda_test = train_test_split(df_lambda, test_size=0.20, random_state=42)
delta_train, delta_test = train_test_split(df_delta, test_size=0.20, random_state=42)

df_delta.head(100)

,delta,main_maximum
0,0.324040,0.488
1,0.320000,0.378
2,0.322020,0.432
3,0.326061,0.542
4,0.328081,0.598
...,...,...
70,0.511919,-0.132
71,0.515960,-0.192
72,0.509899,0.126
73,0.513939,-0.094


,lambda,main_maximum
55,0.183333,-0.530
88,0.231818,-0.856
26,0.140909,-0.236
42,0.162121,-0.384
69,0.203030,-0.664
...,...,...
60,0.195455,-0.612
71,0.207576,-0.694
14,0.119697,-0.084
92,0.242424,-0.926


In [5]:
feature_columns_A = []
no_of_max = tf.feature_column.numeric_column("main_maximum")
feature_columns_A.append(no_of_max)
my_feature_layer_A = tf.keras.layers.DenseFeatures(feature_columns_A)

In [6]:
learning_rate = 1e-3
epochs = 150
batch_size = 30

#specify the classification threshold
classification_threshold = 0.15

# Establish the metrics the model will measure.
metric = [tf.keras.metrics.MeanAbsoluteError()]
layers=[10,12,8]


label_delta="delta"
label_lambda="lambda"


my_model_lambda = create_model2(learning_rate,layers, my_feature_layer_A,metric,my_act_function="relu")
my_model_delta = create_model2(learning_rate,layers, my_feature_layer_A,metric,my_act_function="relu")


In [7]:
epochs_lambda, hist_lambda = train_model(my_model_lambda, lambda_train, epochs, 
                          label_lambda, batch_size)

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


2022-05-12 12:40:59.547613: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 12:40:59.569391: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz


Consider rewriting this model with the Functional API.


In [ ]:
epochs_delta, hist_delta = train_model(my_model_delta, delta_train, epochs, 
                          label_delta, batch_size)